In [ ]:
import os
import shutil

hospital_name = "3_bucheon_hsecg" + "/"
hsecg_data_dir = "./hsecg_data/" + hospital_name
hsecg_label_dir = "./hsecg_label/" + hospital_name
hsecg_data_jpg_dir = "./hsecg_data/" + "3_bucheon_hsecg_jpg" + "/"

In [ ]:
# open excel file
from openpyxl import load_workbook

wb_file_name = "3_bucheon_hsecg"

wb = load_workbook(wb_file_name + ".xlsx" , data_only=True )
ws = wb.active

# print excel file (엑셀 내용 출력)
row_num = 1 # excel row number
for row in ws.values:
    print(row_num, "\t", end='')
    for i in range(len(row)):
        print(row[i], "\t", end='')
    row_num += 1

In [ ]:
# pdf 파일 있는지 체크
for ef in os.listdir(hsecg_data_dir):
    sub_dir = os.path.join(hsecg_data_dir, ef)
    if len(os.listdir(sub_dir)) == 0:
            print("no file exists", ef) # 폴더에 파일이 아예 없다면...
    pdf_file_num = 0
    for file in os.listdir(sub_dir):
        if file[-3:] == "pdf":
            pdf_file_num += 1
            #print(file[-3:])
        else:
            print("not a pdf file", file) # 파일 확장명이 pdf가 아니면 파일명 출력
        if file == os.listdir(sub_dir)[-1] and pdf_file_num > 1:
            print("not 1", ef) # pdf 파일이 1개가 아니라면 출력

In [ ]:
# jpg 파일이 있는지 체크
for ef in os.listdir(hsecg_data_dir):
    sub_dir = os.path.join(hsecg_data_dir, ef)
    if len(os.listdir(sub_dir)) == 0:
            print("no file exists", ef) # 폴더에 파일이 아예 없다면...
    jpg_file_num = 0
    for file in os.listdir(sub_dir):
        if file.rsplit('.')[1].lower() == "jpg" or file.rsplit('.')[1].lower() == "jpeg":
            jpg_file_num += 1
            #print(file[-3:])
        else:
            print("not a jpg file", file) # 파일 확장명이 pdf가 아니면 파일명 출력
        if file == os.listdir(sub_dir)[-1] and jpg_file_num != 1:
            print("not 1", ef) # 파일이 1개가 아니라면 출력

In [ ]:
# jpg 파일이 있는지 체크
for ef in os.listdir(hsecg_data_dir):
    sub_dir = os.path.join(hsecg_data_dir, ef)
    if len(os.listdir(sub_dir)) == 0:
            print("no file exists", ef) # 폴더에 파일이 아예 없다면...
    jpg_file_num = 0
    for file in os.listdir(sub_dir):
        if file.rsplit('.')[1].lower() == "jpg" or file.rsplit('.')[1].lower() == "jpeg":
            file_dir = os.path.join(sub_dir, file)
            print("move" + sub_dir + "/" + file + " to " + hsecg_data_jpg_dir + file)
            shutil.move(sub_dir + "/" + file, hsecg_data_jpg_dir + file)

In [ ]:
#json 파일 있는지 체크
for ef in os.listdir(hsecg_label_dir):
    sub_dir = os.path.join(hsecg_label_dir, ef)
    if len(os.listdir(sub_dir)) == 0:
            print("no file exists", ef) # 폴더에 파일이 아예 없다면...
    json_file_num = 0
    for file in os.listdir(sub_dir):
        if file[-4:] == "json":
            json_file_num += 1
            #print(file[-4:])
        else:
            print("not a json file", ef, file) # 파일 확장명이 json이 아니면 폴더명, 파일명 출력
        if file == os.listdir(sub_dir)[-1] and json_file_num != 1:
            print("not 1 jsons", ef) # 파일이 1개가 아니라면 출력
        if os.listdir(sub_dir)[0] != os.listdir(sub_dir)[-1]:
            print("not 1 files", ef)

In [ ]:
os.listdir(hsecg_label_dir)[0]

In [ ]:
# 과연 라벨링된 번호와 실제 심전도 진단명은 일치할까요...?

from PIL import Image

import pytesseract
import json

# If you don't have tesseract executable in your PATH, include the following:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
# Example tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract'

#ecg_dignosis_name = ["sinus rhythm", "atrial fibrillation", "bundle branch block"]
ecg_dignosis_name = ["sinus", "atrial fibrillation", "bundle branch block"]
ecg_label_dic = {"sinus rhythm":1, "atrial fibrillation":2, "bundle branch block":3, "others":4, "cannot analyze":5}

row_num = 2
for i in range(len(os.listdir(hsecg_data_dir))):
    #print(os.listdir(hsecg_data_dir)[i]) # hsecg_00001_01
    #print(os.listdir(hsecg_label_dir)[i]) # hsecg_00001_01
    df = os.listdir(hsecg_data_dir)[i]
    lf = os.listdir(hsecg_label_dir)[i]
    data_sub_dir = os.path.join(hsecg_data_dir, df)
    label_sub_dir = os.path.join(hsecg_label_dir, lf)
    #print(data_sub_dir) # ./hsecg_data/1_yongin_hsecg/hsecg_00001_01
    #print(label_sub_dir) # ./hsecg_label/1_yongin_hsecg/hsecg_00001_01
    
    
    final_dignosis = ""
    full_dignosis = ""
    
    img_to_txt = pytesseract.image_to_string(Image.open(data_sub_dir + "/" + df + ".jpg"))
    for dignosis in ecg_dignosis_name:
        if (final_dignosis == "") and (dignosis in img_to_txt.lower()) and not("has replaced" in img_to_txt.lower()):
            print("file name =", df, ",", dignosis, "in", img_to_txt)
            final_dignosis = dignosis
            full_dignosis = img_to_txt
    if final_dignosis == "sinus":
        final_dignosis = "sinus rhythm"
    elif final_dignosis == "":
        final_dignosis = "others"
        print("file name =", df, ",", final_dignosis)
        
    pdf_label = ecg_label_dic[final_dignosis]
    print("pdf_label=", pdf_label) # pdf_label= 1
    
    # json 파일 읽기
    with open(label_sub_dir + "/" + lf + ".json", "r", encoding="utf8") as f:
        contents = f.read()
        json_data = json.loads(contents)
    json_label = json_data["labels"]
    print("json_label=", json_label) # json_label= 1
    
    if pdf_label != json_label:
        print("labels do not match", df)
        ws.cell(row=row_num, column=1, value=df)
        ws.cell(row=row_num, column=2, value=pdf_label)
        ws.cell(row=row_num, column=3, value=json_label)
        ws.cell(row=row_num, column=4, value=full_dignosis)
        row_num += 1

In [ ]:
# excel 내용 다시 출력하여 수정사항 체크
row_num = 1 # excel row number
for row in ws.values:
    print(row_num, end="\t")
    for i in range(len(row)):
        print(row[i], end='\t')
    print()
    row_num += 1

In [ ]:
# save excel file (수정한 것을 적용)
wb.save(wb_file_name + "_완료.xlsx")

In [ ]:
from PIL import Image

import pytesseract

# If you don't have tesseract executable in your PATH, include the following:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
# Example tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract'

# Simple image to string
print(pytesseract.image_to_string(Image.open(hsecg_data_jpg_dir + 'hsecg_04253_01.jpg')))

잘 안되네요... 그냥 출력해서 봅시다

In [ ]:
# json label에 5 cannot analyze가 있는지 봅시다...

import json

for i in range(len(os.listdir(hsecg_label_dir))):
    lf = os.listdir(hsecg_label_dir)[i]
    label_sub_dir = os.path.join(hsecg_label_dir, lf)
    with open(label_sub_dir + "/" + lf + ".json", "r", encoding="utf8") as f:
        contents = f.read() # string 타입
        json_data = json.loads(contents)
    ecg_label = json_data["labels"]
    print(lf, ecg_label)
    print()